### 하나은행 예금 데이터 수집
- requests, bs4 패키지를 사용하여 하나은행 예금 데이터를 아래와 같이 수집하세요.
    - get_list, detail_datas 함수를 완성하세요.
    - 상품에 대한 금리는 최고 금리가 출력되도록 하세요.
    - items_df : 상품 리스트에 대한 데이터 프레임
    - items_detail_df : 상품에 대한 상세 정보를 가지는 데이터 프레임
- URL : https://www.kebhana.com/cont/mall/mall08/mall0805/index.jsp?_menuNo=62608

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# 예금 데이터 수집 함수
def get_list():
    url = "https://www.kebhana.com/cont/mall/mall08/mall0805/index.jsp?_menuNo=62608"
    response = requests.get(url)
    dom = BeautifulSoup(response.content, 'html.parser')
    
    dom.select("product-tit")
    
    datas = dom.select(".label-area")

    hana = []

    for data in datas:
        badge = data.select_one("i").text
        title = data.select_one("em a").text
        desc = data.select_one(".tit-desc a").text
        link = "https://www.kebhana.com/" + data.select_one("em a")["href"]
        hana.append({
            "badge":badge,
            "title":title,
            "desc":desc,
            "link":link
        })

    hana_df = pd.DataFrame(hana)
    
    return hana_df

In [3]:
# 예금 데이터 상세페이지 수집 함수
def detail_datas(d_link, idx):
    driver = webdriver.Chrome()
    driver.get(d_link)
    driver.find_elements_by_css_selector(".prodcutInfo")
    
    leng = len(driver.find_elements_by_css_selector("#contents > div.mall-detail-content > dl > dt"))
    dic = []
    for num in range(leng):
        title = driver.find_elements_by_css_selector("#contents > div.mall-detail-content > dl > dt")[num].text.replace("\n"," ")
        if title != "금리":
            dic.append({
                "index_txt" : idx,
                "title_txt" : title,
                "text_txt" : driver.find_elements_by_css_selector("#contents > div.mall-detail-content > dl > dd")[num].text.replace("\n"," ")
            })
            
        else:
                dic.append({
                    "index_txt" : idx,
                    "title_txt" : driver.find_elements_by_css_selector("#contents > div.mall-detail-content > dl > dt")[num].text,           
                    "text_txt" : None
                })

    driver.quit()
    return pd.DataFrame(dic)

In [4]:
items_df = get_list()

In [6]:
dfs = []
for idx, data in items_df.iterrows():
    dfs.append(detail_datas(data.link, idx))
    print(idx, sep='\n')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [7]:
item_details_df = pd.concat(dfs)
item_details_df.reset_index(drop=True, inplace=True)

In [8]:
items_df.tail()

,badge,title,desc,link
10,정기예금,정기예금,"목돈을 일정기간동안 예치하여 안정적인 수익을 추구하는 예금으로서, 이자획득을 목적으...",https://www.kebhana.com//cont/mall/mall08/mall...
11,정기예금,환매조건부매도증권(RP),하나은행이 일정기간 후 일정가액으로 환매수 할 것을 조건으로 고객에게 채권을 매도하...,https://www.kebhana.com//cont/mall/mall08/mall...
12,정기예금,양도성예금증서(CD),무기명 할인식 또는 등록발행 가능한 증서로서 예치일로부터 30일이상 5년이내의 기간...,https://www.kebhana.com//cont/mall/mall08/mall...
13,정기예금,표지어음,은행이 보유하고 있는 원어음(상업어음 또는 무역어음)을 근거로 발행하는 배서양도가 ...,https://www.kebhana.com//cont/mall/mall08/mall...
14,정기예금,1년 연동형 정기예금,서울보증보험의 보증서 발급 담보용 정기예금,https://www.kebhana.com//cont/mall/mall08/mall...


In [9]:
item_details_df[:20]

,index_txt,title_txt,text_txt
0,0,상품특징,(스마트폰 전용 예금) 마케팅동의와 하나오픈뱅킹 서비스를 통하여 상품 신규 시 우대...
1,0,가입대상,실명의 개인 및 개인사업자
2,0,예금종류,정기예금
3,0,가입기간,1년
4,0,가입금액,"1백만원 이상 ~ 3천만원 이내 (단, 오픈뱅킹 1일 이체한도는 1천만원)"
5,0,이자지급방법,만기일시지급식
6,0,금리,None
7,0,우대금리,"최대 연 0.2% (2019.12.18 기준, 세전) 이 예금의 가입자가 아래 항목..."
8,0,특별금리,이 예금 가입시 하나오픈뱅킹서비스 출금이체(다른은행 계좌)로 신규한 경우 0.2%추...
9,0,상품내용 변경에 관한 사항,상품내용 변경에 관한 사항 (2020.07.01 변경) <중도해지금리변경> 변경 전...


In [10]:
# 상품별 최고 금리
item_details_df[item_details_df["title_txt"] == "금리"]

,index_txt,title_txt,text_txt
6,0,금리,None
20,1,금리,None
34,2,금리,None
48,3,금리,None
60,4,금리,None
73,5,금리,None
85,6,금리,None
98,7,금리,None
110,8,금리,None
122,9,금리,None
